In [1]:
import numpy as np
import torch
from itertools import product
from collections import defaultdict
from config import Config
config = Config()
# from util import solve_chiN, solve_chiM
def solve_chiN(I, xi, sigma_u, sigma_v, theta, tol=1e-12, max_iter=10000):
    """
    Solve for the noncollusive slope chi^N in the Kyle-type model.

    We use the 3-equation system from Section 3.2 in the paper:
      (1) chi^N = 1 / [(I+1)*lambda^N]
      (2) lambda^N = [theta * gamma^N + xi] / (theta + xi^2)
      (3) gamma^N = (I*chi^N) / [(I*chi^N)^2 + (sigma_u/sigma_v)^2]

    We do simple fixed-point iteration over chi^N.

    Returns:
      float: chi^N
      float: lambda^N
    """
    chi = 0.1  # Arbitrary initial guess
    for _ in range(max_iter):
        # Given chi, compute gamma^N:
        gamma = (I * chi) / ((I * chi)**2 + (sigma_u / sigma_v)**2)

        # Then lambda^N:
        lam = (theta * gamma + xi) / (theta + xi**2)

        # Then the updated chi^N:
        new_chi = 1.0 / ((I + 1) * lam)
        # print(new_chi)
        if abs(new_chi - chi) < tol:
            return new_chi, lam
        chi = new_chi

    raise RuntimeError("solve_chiN did not converge within max_iter")

def solve_chiM(I, xi, sigma_u, sigma_v, theta, tol=1e-12, max_iter=10000):
    """
    Solve for the *perfect-collusion* slope chi^M in the Kyle-type model.

    From Section 3.3 in the paper:
      (1) chi^M = 1 / [2*I * lambda^M]
      (2) lambda^M = [theta * gamma^M + xi] / (theta + xi^2)
      (3) gamma^M = (I*chi^M) / [(I*chi^M)^2 + (sigma_u/sigma_v)^2]

    Similar fixed-point iteration as above.
    """
    chi = 0.1  # Arbitrary initial guess
    for _ in range(max_iter):
        gamma = (I * chi) / ((I * chi)**2 + (sigma_u / sigma_v)**2)
        lam = (theta * gamma + xi) / (theta + xi**2)
        new_chi = 1.0 / (2.0 * I * lam)
        # print(new_chi)
        if abs(new_chi - chi) < tol:
            return new_chi, lam
        chi = new_chi

    raise RuntimeError("solve_chiM did not converge within max_iter")

In [2]:

class Q_table_Batch:

    def __init__(self,
                 config,
                 B = 1000):
        self.Np = range(config.Np)
        self.Nv = range(config.Nv)
        self.B = B
        self.states = list(product(self.Np, self.Nv))

        # self.Q = {s:np.zeros(config.Nx) for s in self.states}
        self.Q = torch.zeros((B,config.I, config.Np, config.Nv, config.Nx), dtype = torch.float32)
        
    def get_Q_value(self, state, action):
        p, v = state
        return self.Q[..., p, v, action]

    def get_best_action(self, state):
        p, v = state[:, 0], state[:, 1] # Bx1, Bx1
        return torch.argmax(self.Q[torch.arange(self.B),:, p, v], axis = -1) # B x I

    def get_best_value(self, state):
        p, v = state
        return torch.max(self.Q[..., p, v])

    def update(self, state, action, value):
        p, v = state
        self.Q[..., p, v, action] = value

class InformedAgents_Batch:

    def __init__(self, config, B = 1000):
        
        # parameters
        self.n_actions = config.Nx
        self.Np = config.Np
        self.Nv = config.Nv
        self.n_states = self.Np * self.Nv

        self.rho = config.rho
        self.alpha = config.alpha
        self.beta = config.beta

        self.sigma_v = config.sigma_v
        self.v_bar = config.v_bar
        self.sigma_u = config.sigma_u
        self.xi = config.xi
        self.theta = config.theta
        self.iota = config.iota
        self.I = config.I
        self.B = B
        # Q-table for RL
        self.Q = Q_table_Batch(config)

        # state count dictionary for epsilon decay
        # self.state_count = defaultdict(int)
        self.state_count = torch.zeros((B, self.I, self.Np, self.Nv), dtype = torch.int16)

        # convergence dictionary
        self.last_optimal = torch.zeros((B, self.I, self.Np, self.Nv), dtype = torch.int16)
        self.convergence_counter = torch.zeros((B, self.I), dtype = torch.int16)

        # discretization of states
        self.get_discrete_states()

        # initialize Q-table
        self.initialize_Q()

    
    def get_epsilon(self, state):
        p, v = state[:, 0], state[:, 1] # Bx1, Bx1
        count = self.state_count[torch.arange(self.B), :, p, v] # B x I
        # print(count.shape)
        # print(count)
        self.state_count[torch.arange(self.B),:, p, v] += 1
        return np.exp(-self.beta * count) # B x I
    
    def get_action(self, state):
        epsilon = self.get_epsilon(state) # B x I
        # print(epsilon.shape)
        greedy = torch.randn(self.B, self.I) > epsilon
        optimal_action = self.Q.get_best_action(state) # B x I
        print(optimal_action.max())
        self.check_convergence(state, optimal_action)
        return torch.where(greedy, optimal_action, torch.randint(0, self.n_actions, (self.B, self.I)))
        torch.randint()
        # if torch.randn(self.B, self.I) < epsilon:
        #     return torch.randint(self.n_actions, (self.B, self.I))
        #     return np.random.randint(self.n_actions)
        # else:
        #     optimal_action = self.Q.get_best_action(state)
        #     self.check_convergence(state, optimal_action)
        #     return optimal_action
        
    def update(self, state, action, reward, next_state):
        learning = self.alpha * (reward + self.rho * self.Q.get_best_value(next_state)) # B x I
        memory = (1 - self.alpha) * self.Q.get_Q_value(state, action) # B x I
        value = learning + memory # B x I
        self.Q.update(state, action, value) 
    
    def get_grid_point_values_v(self):
        """
        Returns a zero indexed dictionary of the grid points for the state space of v
        """
        standard_normal = torch.distributions.Normal(0, 1)
        grid_point = [(2 * k - 1) / (2 * self.Nv) for k in range(1, self.Nv + 1)]
        values = standard_normal.icdf(torch.tensor(grid_point))
        return torch.tensor([self.v_bar + self.sigma_v * value for value in values], dtype = torch.float32)
    
    def get_grid_point_values_x(self):
        self.chiN, self.lambdaN = solve_chiN(I = self.I, xi = self.xi, sigma_u = self.sigma_u, sigma_v = self.sigma_v, theta = self.theta)
        self.chiM, self.lambdaM = solve_chiM(I = self.I, xi = self.xi, sigma_u = self.sigma_u, sigma_v = self.sigma_v, theta = self.theta)
        self.x_n, self.x_m = self.chiN, self.chiM # assuming v - v_bar = 1
        span_x = abs(self.x_n - self.x_m)
        low, high = -max(self.x_n, self.x_m) - self.iota * span_x, max(self.x_n, self.x_m) + self.iota * span_x
        values = np.linspace(low, high, self.n_actions)
        values = torch.tensor(values, dtype = torch.float32)
        # return {idx: float(val) for idx, val in enumerate(values)}
        return values
    
    def get_grid_point_values_p(self):
        try:
            lambda_for_p = max(self.lambdaN, self.lambdaM)
        except:
            raise ValueError("Call get_grid_point_values_x() first")
        ph = self.v_bar + lambda_for_p * (self.I * max(self.x_n, self.x_m) + self.sigma_u * 1.96)
        pl = self.v_bar - lambda_for_p * (self.I * max(self.x_n, self.x_m) + self.sigma_u * 1.96)
        span_p = ph - pl
        values = np.linspace(pl - self.iota * span_p, ph + self.iota * span_p, self.Np)
        return torch.tensor(values, dtype = torch.float32)
    
    def get_discrete_states(self):
        self.v_discrete = self.get_grid_point_values_v()
        self.x_discrete = self.get_grid_point_values_x()
        self.p_discrete = self.get_grid_point_values_p()

    def continuous_to_discrete(self, p, v):
        p_idx = min(self.p_discrete, key=lambda x: abs(self.p_discrete[x] - p))
        v_idx = min(self.v_discrete, key=lambda x: abs(self.v_discrete[x] - v))
        return p_idx, v_idx

    def initialize_Q(self):
        for p in range(self.Np):
            for v in range(self.Nv):
                state = (p, v)
                for x in range(self.n_actions):
                    value = 0
                    for x_i in range(self.n_actions):
                        value += self.v_discrete[v] - (self.v_bar + self.lambdaN * (self.x_n + (self.I - 1) * self.x_discrete[x_i]))
                    value *= self.x_discrete[x] / ((1 - self.rho) * self.n_actions)
                    self.Q.update(state, x, value)

    def check_convergence(self, state, action):
        # print(self.last_optimal)
        p, v = state[:, 0], state[:, 1] # Bx1, Bx1

        self.convergence_counter = torch.where(action != self.last_optimal[torch.arange(self.B),:, p, v], 
                                               0, 
                                               self.convergence_counter + 1)


        # if action != self.last_optimal[]
        #     self.last_optimal[state] = action
        #     self.convergence_counter = 0

        # else:
        #     self.convergence_counter += 1

In [3]:

class CircularBuffer_Batch:
    """
    Circular buffer for storing historical data.
    """
    def __init__(self, size, B = 1000):
        self.size = size
        self.buffer = torch.zeros((B, size), dtype = torch.float32)
        self.index = 0

    def add(self, value):
        self.buffer[:, self.index] = value # B x 1
        self.index = (self.index + 1) % self.size

    def get(self):
        return torch.cat((self.buffer[:, self.index:], self.buffer[:, :self.index]), dim = 1) # B x size
        # return np.concatenate((self.buffer[self.index:], self.buffer[:self.index]))

class AdaptiveMarketMaker_Batch:

    def __init__(self, config):
        self.theta = config.theta
        self.Tm = config.Tm

        self.vars_ = ['v','p','z','y']
        self.historical_data = {var: CircularBuffer_Batch(size = self.Tm) for var in self.vars_}
    def OLS(self, y, X):
        """
        Perform batched Ordinary Least Squares (OLS) regression using PyTorch.
        Assumes X and y are both (B x Tm).
        """
        y = y.get()  # B x Tm
        X = X.get()  # B x Tm

        B, T = X.shape

        # Add intercept: (B x T x 2) where last dim is [x_t, 1]
        X_aug = torch.stack([X, torch.ones_like(X)], dim=2)  # B x T x 2

        # Reshape y to (B x T x 1)
        y = y.unsqueeze(-1)  # B x T x 1

        # Batched least squares using pseudo-inverse: beta = (X^T X)^-1 X^T y
        X_T = X_aug.transpose(1, 2)  # B x 2 x T
        beta = torch.linalg.pinv(X_T @ X_aug) @ X_T @ y  # B x 2 x 1

        return beta.squeeze(-1)  # B x 2

    
    def determine_price(self, yt):
        """
        Determines the price based on historical data and a given input.
        This method uses Ordinary Least Squares (OLS) regression to calculate
        yt (float): The input value for which the price needs to be determined.
        Returns:
        float: The determined price based on the input `yt`.
        """

        xi_1, _ = self.OLS(self.historical_data['z'], self.historical_data['p']) # B x 2 slope intercept
        gamma_1, gamma_0 = self.OLS(self.historical_data['v'], self.historical_data['y']) # B x2
        lambda_ = (xi_1 + self.theta * gamma_1) / (xi_1**2 + self.theta) # B x 1
        # print(lambda_)
        price = gamma_0 + lambda_ * yt # B x 1 + B x 1 * scalar = B x 1
        return price.squeeze(-1) # B x 1 -> B
    
    def update(self, vt, pt, zt, yt):
        """
        Updates the historical data with the given batched values.
        Parameters:
        vt (torch.Tensor): Batched values of `v` at time `t` with shape (B,).
        pt (torch.Tensor): Batched values of `p` at time `t` with shape (B,).
        zt (torch.Tensor): Batched values of `z` at time `t` with shape (B,).
        yt (torch.Tensor): Batched values of `y` at time `t` with shape (B,).
        """
        for var, value in zip(self.vars_, [vt, pt, zt, yt]):
            self.historical_data[var].add(value)

In [4]:
class PreferredHabitatAgent:

    def __init__(self, config):
        self.xi = config.xi
        self.v_bar = config.v_bar

    def get_action(self, pt):
        z = -self.xi * (pt - self.v_bar)
        return z

In [1]:
from agents import *
from config import Config
config = Config()

In [2]:
m = AdaptiveMarketMaker_Batch(config)
x = CircularBuffer_Batch(1000)
y = CircularBuffer_Batch(1000)
u = torch.randn(1000)
u = 1.2032 + 0.1 * u
for i in range(1000):
    x.add(torch.randn(1000)*10+3)
    y.add(torch.randn(1000))
print(m.OLS(x, y))

tensor([[ 0.0259,  2.8641],
        [-0.2565,  3.0549],
        [ 0.0686,  2.7330],
        ...,
        [-0.2709,  3.5111],
        [-0.0512,  2.9236],
        [-0.1282,  2.7389]])


In [23]:
ib = InformedAgents_Batch(config)


In [24]:
q = Q_table_Batch(config)
q.Q.shape

torch.Size([1000, 2, 31, 10, 15])

In [5]:
for i in range(100):
    _p_init = torch.randint(0, 31, (1000,))
    _v_init = torch.randint(0, 10, (1000,))

    state = torch.stack((_p_init, _v_init), dim=1) # B x 2
    action = ib.get_action(state) # B x I
    # print(action)

In [6]:
_v = ib.v_discrete[state[:,1]]

In [7]:
y=ib.x_discrete[action].sum(axis = -1) 
pt = m.determine_price(y)

In [8]:
hab = PreferredHabitatAgent(config)
z = hab.get_action(pt)
z

tensor([500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500., 500.,
        500., 500., 500., 500., 500., 50

In [9]:
m.update(ib.v_discrete[state[:,1]], ib.p_discrete[state[:,0]], z, y)

In [10]:
m.historical_data['v'].get()

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.6449],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0364],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.6449],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.6147],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.8743],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.6147]])

In [11]:
v,p = ib.continuous_to_discrete(ib.v_discrete[state[:,1]], ib.p_discrete[state[:,0]])
v.shape

torch.Size([1000])

In [12]:
action.shape

torch.Size([1000, 2])

In [13]:
reward = ((_v - pt)* action.T).T

In [14]:
v = ib.Q.get_best_value(state)
print(v)

tensor([[6758.8027, 6758.8027],
        [4680.0347, 4680.0347],
        [6758.8027, 6758.8027],
        ...,
        [2455.3997, 2455.3997],
        [1568.2314, 1568.2314],
        [2455.3997, 2455.3997]])


In [26]:
ib.convergence_counter.min()

tensor(0, dtype=torch.int16)

In [20]:
ib.update(state, action, reward, state)

In [17]:
ib.Q.Q

tensor([[[[[ 6758.8027,  5793.2598,  4827.7163,  ..., -4827.7163,
            -5793.2598, -6758.8027],
           [ 4680.0347,  4011.4585,  3342.8821,  ..., -3342.8821,
            -4011.4585, -4680.0347],
           [ 3443.3943,  2951.4807,  2459.5674,  ..., -2459.5674,
            -2951.4807, -3443.3943],
           ...,
           [-1165.6176,  -999.1008,  -832.5840,  ...,   832.5840,
              999.1008,  1165.6176],
           [-2402.2590, -2059.0793, -1715.8994,  ...,  1715.8994,
             2059.0793,  2402.2590],
           [-4481.0259, -3840.8796, -3200.7329,  ...,  3200.7329,
             3840.8796,  4481.0259]],

          [[ 6758.8027,  5793.2598,  4827.7163,  ..., -4827.7163,
            -5793.2598, -6758.8027],
           [ 4680.0347,  4011.4585,  3342.8821,  ..., -3342.8821,
            -4011.4585, -4680.0347],
           [ 3443.3943,  2951.4807,  2459.5674,  ..., -2459.5674,
            -2951.4807, -3443.3943],
           ...,
           [-1165.6176,  -999.1008,  -8

In [ ]:
b_idx, i_idx = torch.meshgrid(
    torch.arange(B, device=device),
    torch.arange(I, device=device),
    indexing='ij'
)

# now index into Q along all dims at once
self.Q[b_idx, i_idx, p[:, None], v[:, None], action] = updated

In [106]:
torch.stack([torch.zeros(config.Nx) for _ in range(config.Np * config.Nv) for _ in range(10)]).to(torch.float32)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [107]:
ib.x_discrete[[0,1,2,3]]

tensor([-170.8333, -146.4285, -122.0238,  -97.6190])

In [21]:
ib.v_discrete

tensor([-0.6450, -0.0364,  0.3254,  0.6147,  0.8745,  1.1260,  1.3857,  1.6748,
         2.0371,  2.6445], dtype=torch.float16)

In [22]:
ib.p_discrete

tensor([0.1996, 0.2529, 0.3062, 0.3596, 0.4131, 0.4663, 0.5195, 0.5732, 0.6265,
        0.6797, 0.7334, 0.7866, 0.8398, 0.8931, 0.9468, 1.0000, 1.0537, 1.1064,
        1.1602, 1.2139, 1.2666, 1.3203, 1.3740, 1.4268, 1.4805, 1.5332, 1.5869,
        1.6406, 1.6934, 1.7471, 1.8008], dtype=torch.float16)

In [24]:
ib.Q.Q.shape

torch.Size([1000, 2, 31, 10, 15])

In [1]:
from util import *
from config import Config
config = Config(steps= 50)
log, agents = simulate_batch(T=50,config=config, save_path = 'tes_2.pt', continue_simulation='test.pt')

/Users/moyuanchen/Documents/thesis/collusion/util.py:187: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  log = torch.load(continue_simulation)


In [2]:
data = torch.load('test.pt')

/var/folders/ld/776wkv1x7j163s0ynxr2xqq00000gn/T/ipykernel_75028/14747726.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('test.pt')


In [ ]:
ib = InformedAgents_Batch(config)


In [3]:
data

{'v': tensor([[2, 8, 7,  ..., 0, 6, 9],
         [1, 6, 8,  ..., 0, 1, 1],
         [3, 2, 0,  ..., 6, 8, 9],
         ...,
         [6, 9, 7,  ..., 6, 8, 4],
         [8, 5, 4,  ..., 6, 4, 9],
         [0, 3, 0,  ..., 4, 7, 9]]),
 'p': tensor([[0.4131, 0.1996, 0.1996,  ..., 0.1996, 0.1996, 0.0000],
         [0.4663, 0.1996, 0.1996,  ..., 0.1996, 0.1996, 0.0000],
         [1.2666, 0.1996, 0.1996,  ..., 0.1996, 0.1996, 0.0000],
         ...,
         [1.4268, 0.1996, 0.1996,  ..., 0.1996, 0.1996, 0.0000],
         [0.2529, 0.1996, 0.1996,  ..., 0.1996, 0.1996, 0.0000],
         [1.2666, 0.1996, 0.5732,  ..., 0.1996, 0.1996, 0.0000]],
        dtype=torch.float16),
 'z': tensor([[500.0000, 398.7500, 571.5000,  ..., 453.5000, 510.7500,   0.0000],
         [500.0000, 499.5000, 499.5000,  ..., 570.0000, 511.2500,   0.0000],
         [500.0000, 529.5000, 689.0000,  ..., 537.0000, 503.2500,   0.0000],
         ...,
         [500.0000, 463.7500, 876.0000,  ..., 451.5000, 434.5000,   0.0000],
  